In [21]:
import pandas as pd
import datetime as dt
import info_collection.ship_usage_data.average_load as avg

In [22]:
# Load the trips data
trips_df = pd.read_csv("ferry_tips_data.csv")

# look for specific route (for example 17)
#selected_route = trips_df[trips_df["route_id"] == 17]

# Convert "time_departure" to a specific format ig? 
#tbh i think time _departure is called something else but im lazy so i will check later
#selected_route["time_departure"] = pd.to_datetime(selected_route["time_departure"])

# Create a "date" column for grouping
#selected_route["date"] = selected_route["time_departure"].dt.date

# Group data to calculate hourly load (pedestrians, cars, etc)


In [23]:
# Define variables according to different ferries 
ROUND_TRIP_TIME = 20  # Time for a round trip in minutes
PEAK_TIMES = 0.8  # Load above 80% of capacity is considered peak maybe hmm
OFF_PEAK_TIME = 0.4  # Load between 40% and 80% is off-peak
SHIP_CAPACITY = 50  # Example capacity (will be different obv idk what to put here rn lol)
CUTOFF_TIME = 10  # Ticket cutoff time in minutes before departure
ROUTE_ID = 17

# Initialize an empty schedule list
schedule = []


In [25]:
# Iterate through each hour to calculate demand and schedule the times accordingly 
# at first i didnt use unique but ig there will be different ferries that have the same time yk??
#idk if this makes sense 
for day_of_week in range(7):
    daily_data = avg.average_load_by_hour(ROUTE_ID, day_of_week, trips_df)
    print(daily_data)
    for hour in daily_data["hour"].unique():
    
        # Calculate total load for this hour
        total_load = daily_data["cars_outbound"]

        # Determine trip frequency based on load
        if total_load >= SHIP_CAPACITY * PEAK_TIMES:
            # High demand: Schedule trips every RRT
            trips = 60 // ROUND_TRIP_TIME
            for trip in range(trips):
                departure_time = dt.time(hour=hour, minute=trip * ROUND_TRIP_TIME)
                schedule.append({"date": day_of_week, "hour": hour, "departure_time": departure_time})
        elif total_load >= SHIP_CAPACITY * OFF_PEAK_TIME:
            # Medium demand: Schedule fewer trips (e.g., every 40-50 minutes?)
            trips = 60 // 40
            for trip in range(trips):
                departure_time = dt.time(hour=hour, minute=trip * 40)
                schedule.append({"date": day_of_week, "hour": hour, "departure_time": departure_time})
        elif total_load > 0:
            # Low demand
            if hour % 2 == 0:  # Schedule trips every 2 hours ?? can be changed obv
                departure_time = dt.time(hour=hour, minute=0)
                schedule.append({"date": day_of_week, "hour": hour, "departure_time": departure_time})


    hour  route_id  ferry_id  cars_outbound  trucks_outbound  \
0      5      17.0     289.0       8.000000         0.062500   
1      6      17.0     289.0       8.083916         0.188811   
2      7      17.0     289.0       8.436242         0.268456   
3      8      17.0     289.0       8.456376         0.463087   
4      9      17.0     289.0       8.144737         0.671053   
5     10      17.0     289.0       6.125828         0.662252   
6     11      17.0     289.0       7.171053         0.671053   
7     12      17.0     289.0       6.064677         0.582090   
8     13      17.0     289.0       6.555000         0.475000   
9     14      17.0     289.0       7.661692         0.572139   
10    15      17.0     289.0      11.973856         0.679739   
11    16      17.0     289.0      13.650000         0.650000   
12    17      17.0     289.0      11.425000         0.335000   
13    18      17.0     289.0       6.368421         0.282895   
14    19      17.0     289.0       4.211

/Users/chapman/Documents/hackathon/hack-a-fleet/info_collection/ship_usage_data/average_load.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_df["weekday"] = route_df["time_departure"].apply(lambda x: x.weekday())


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().